In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/329afternoon/sent2kaggle/readme.md
/kaggle/input/329afternoon/sent2kaggle/requirements.txt
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/data_process.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/train.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/run.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/metrics.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/task_config.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/model.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/cur_best.pkl
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/data_loader.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/k.txt
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/loss.png
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/config.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/utils.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/task_train.py
/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/only2test.

In [2]:
### hoho

os.getcwd()

'/kaggle/working'

In [3]:
### hoho
class config_demo:
    hoho = 1
    hoho2 = 2
    
config_demo.hoho2

2

In [4]:
RANDOM_STATE = 6666
BERT_MODEL_NAME = 'bert-base-chinese'
ROBERTA_MODEL_NAME = 'hfl/chinese-roberta-wwm-ext'

In [5]:
### config.py

import os
import torch

class config:

    data_dir = os.getcwd() + '/data/clue/'
    # train_dir = data_dir + 'train.npz'
    # test_dir = data_dir + 'test.npz'
    train_dir = '/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/data/clue/train.npz'
    test_dir = '/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/data/clue/test.npz'
    files = ['train', 'test']
    bert_model = BERT_MODEL_NAME #'/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/pretrained_bert_models/bert-base-chinese/'
    roberta_model = ROBERTA_MODEL_NAME #'/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/pretrained_bert_models/chinese_roberta_wwm_large_ext/'
    model_dir = os.getcwd() + '/experiments/clue/'
    log_dir = model_dir + 'train.log'
    case_dir = os.getcwd() + '/case/bad_case.txt'

    # 训练集、验证集划分比例
    dev_split_size = 0.1

    # 是否加载训练好的NER模型
    load_before = False

    # 是否对整个BERT进行fine tuning
    full_fine_tuning = True

    # hyper-parameter
    learning_rate = 3e-5
    weight_decay = 0.01
    clip_grad = 5

    batch_size = 32
    epoch_num = 50
    min_epoch_num = 5
    patience = 0.0002
    patience_num = 10

    gpu = '1'

    if gpu != '':
        device = torch.device(f"cuda")
    else:
        device = torch.device("cpu")

    labels = ['address', 'book', 'company', 'game', 'government',
              'movie', 'name', 'organization', 'position', 'scene']

    label2id = {
        "O": 0,
        "B-address": 1,
        "B-book": 2,
        "B-company": 3,
        'B-game': 4,
        'B-government': 5,
        'B-movie': 6,
        'B-name': 7,
        'B-organization': 8,
        'B-position': 9,
        'B-scene': 10,
        "I-address": 11,
        "I-book": 12,
        "I-company": 13,
        'I-game': 14,
        'I-government': 15,
        'I-movie': 16,
        'I-name': 17,
        'I-organization': 18,
        'I-position': 19,
        'I-scene': 20,
        "S-address": 21,
        "S-book": 22,
        "S-company": 23,
        'S-game': 24,
        'S-government': 25,
        'S-movie': 26,
        'S-name': 27,
        'S-organization': 28,
        'S-position': 29,
        'S-scene': 30
    }

    id2label = {_id: _label for _label, _id in list(label2id.items())}


In [6]:
### hoho

!mkdir experiments
!mkdir case
!mkdir experiments/clue

In [7]:
### data_loader.py

import torch
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import Dataset


class NERDataset(Dataset):
    def __init__(self, words, labels, config, word_pad_idx = 0, label_pad_idx = -1):
        self.tokenizer = BertTokenizer.from_pretrained(config.bert_model, do_lower_case=True)
        self.label2id = config.label2id
        self.id2label = {_id: _label for _label, _id in list(config.label2id.items())}
        self.dataset = self.preprocess(words, labels)
        self.word_pad_idx = word_pad_idx
        self.label_pad_idx = label_pad_idx
        self.device = config.device

    def preprocess(self, origin_sentences, origin_labels):
        """
        Maps tokens and tags to their indices and stores them in the dict data.
        examples: 
            word:['[CLS]', '浙', '商', '银', '行', '企', '业', '信', '贷', '部']
            sentence:([101, 3851, 1555, 7213, 6121, 821, 689, 928, 6587, 6956],
                        array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]))
            label:[3, 13, 13, 13, 0, 0, 0, 0, 0]
        """
        data = []
        sentences = []
        labels = []
        for line in origin_sentences:
            # replace each token by its index
            # we can not use encode_plus because our sentences are aligned to labels in list type
            words = []
            word_lens = []
            for token in line:
                words.append(self.tokenizer.tokenize(token))
                word_lens.append(len(token))     # hoho: w为啥要计算token的长度？难道不是单个字？
            # 变成单个字的列表，开头加上[CLS]
            words = ['[CLS]'] + [item for token in words for item in token]
            token_start_idxs = 1 + np.cumsum([0] + word_lens[:-1])  # 前面加个1，因为第0位位CLS，所以实际开始序号为1
            sentences.append((self.tokenizer.convert_tokens_to_ids(words), token_start_idxs))
        for tag in origin_labels:
            label_id = [self.label2id.get(t) for t in tag]
            labels.append(label_id)
        for sentence, label in zip(sentences, labels):
            data.append((sentence, label))
        return data

    def __getitem__(self, idx):
        """sample data to get batch"""
        word = self.dataset[idx][0]
        label = self.dataset[idx][1]

        return [word, label]

    def __len__(self):
        """get dataset size"""
        return len(self.dataset)

    def collate_fn(self, batch):
        """
        process batch data, including:
            1. padding: 将每个batch的data padding到同一长度（batch中最长的data长度）
            2. aligning: 找到每个sentence sequence里面有label项，文本与label对齐
            3. tensor：转化为tensor
        """
        sentences = [x[0] for x in batch]
        labels = [x[1] for x in batch]

        # batch length
        batch_len = len(sentences)

        # compute length of longest sentence in batch
        max_len = max([len(s[0]) for s in sentences])   # hoho: 注意每个sentence的结构，第0位的位真正的sentence
        max_label_len = 0

        # padding data 初始化
        batch_data = self.word_pad_idx * np.ones((batch_len, max_len))
        batch_label_starts = []

        # padding and aligning
        for j in range(batch_len):
            cur_len = len(sentences[j][0])
            batch_data[j][:cur_len] = sentences[j][0]
            # 找到有标签的数据的index（[CLS]不算）
            label_start_idx = sentences[j][-1]
            label_starts = np.zeros(max_len)
            label_starts[[idx for idx in label_start_idx if idx < max_len]] = 1
            batch_label_starts.append(label_starts)
            max_label_len = max(int(sum(label_starts)), max_label_len)

        # padding label
        batch_labels = self.label_pad_idx * np.ones((batch_len, max_label_len))
        for j in range(batch_len):
            cur_tags_len = len(labels[j])
            batch_labels[j][:cur_tags_len] = labels[j]

        # convert data to torch LongTensors
        batch_data = torch.tensor(batch_data, dtype=torch.long)
        batch_label_starts = torch.tensor(batch_label_starts, dtype=torch.long)
        batch_labels = torch.tensor(batch_labels, dtype=torch.long)

        # shift tensors to GPU if available
        batch_data, batch_label_starts = batch_data.to(self.device), batch_label_starts.to(self.device)
        batch_labels = batch_labels.to(self.device)
        return [batch_data, batch_label_starts, batch_labels]



In [8]:
# hoho
1 + np.cumsum([0] + [1, 1, 1, 1])

array([1, 2, 3, 4, 5])

In [9]:
### data_process.py

import os
import json
import logging
import numpy as np


class Processor:
    def __init__(self, config):
        self.data_dir = config.data_dir
        self.config = config

    def process(self):
        """
        process train and test data
        """
        for file_name in self.config.files:
            self.preprocess(file_name)
        

    def preprocess(self, mode):
        """
        params:
            words：将json文件每一行中的文本分离出来，存储为words列表
            labels：标记文本对应的标签，存储为labels
        examples:
            words示例：['生', '生', '不', '息', 'C', 'S', 'O', 'L']
            labels示例：['O', 'O', 'O', 'O', 'B-game', 'I-game', 'I-game', 'I-game']
        """
#         input_dir = self.data_dir + str(mode) + '.json'
#         output_dir = self.data_dir + str(mode) + '.npz'

        input_dir = '/kaggle/input/329afternoon/sent2kaggle/BERT-LSTM-CRF/data/clue/' + str(mode) + '.json'
        output_dir = '/kaggle/working/' + str(mode) + '.npz'

        if os.path.exists(output_dir) is True:
            return
        word_list = []
        label_list = []
        with open(input_dir, 'r', encoding='utf-8') as f:
            # 先读取到内存中，然后逐行处理
            for line in f.readlines():
                # loads()：用于处理内存中的json对象，strip去除可能存在的空格
                json_line = json.loads(line.strip())

                text = json_line['text']
                words = list(text)
                # 如果没有label，则返回None
                label_entities = json_line.get('label', None)
                labels = ['O'] * len(words)

                if label_entities is not None:
                    for key, value in label_entities.items():
                        for sub_name, sub_index in value.items():
                            for start_index, end_index in sub_index:
                                assert ''.join(words[start_index:end_index + 1]) == sub_name
                                if start_index == end_index:
                                    labels[start_index] = 'S-' + key
                                else:
                                    labels[start_index] = 'B-' + key
                                    labels[start_index + 1:end_index + 1] = ['I-' + key] * (len(sub_name) - 1)
                word_list.append(words)
                label_list.append(labels)
                # 保存成二进制文件
            np.savez_compressed(output_dir, words=word_list, labels=label_list)
            print("--------{} data process DONE!--------".format(mode))

In [10]:
### hoho
processor = Processor(config)
processor.process()

/opt/conda/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


--------train data process DONE!--------
--------test data process DONE!--------


In [11]:
# hoho
# !pip install transformers
# torch.__version__
!pip install pytorch-crf -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [12]:
from transformers.models.bert.modeling_bert import *
from torch.nn.utils.rnn import pad_sequence
from torchcrf import CRF

In [13]:
### model.py

class BertNER(BertPreTrainedModel):
    
    def __init__(self, config):   # 注意：这个config并非自定义的config类
        super(BertNER, self).__init__(config)
        
#         print(f'config: {config}')

        hidden_size = 768  # 这里要跟bert的输入输出size一致
        
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.bilstm = nn.LSTM(
            input_size = hidden_size, #config.lstm_embedding_size,    # 这里要跟bert的输出size一致
            hidden_size = hidden_size // 2, #config.hidden_size // 2,
            batch_first = True,
            num_layers = 2,
            dropout = 0.5, #config.lstm_dropout_prob,
            bidirectional = True
        )
        self.classifier = nn.Linear(hidden_size, config.num_labels)
        self.crf = CRF(config.num_labels, batch_first = True)
        
        self.init_weights()
        
    def forward(self, 
                input_data, token_type_ids = None, attention_mask = None, 
                labels = None, position_ids = None, inputs_embeds = None, head_mask = None):
        input_ids, input_token_starts = input_data
        outputs = self.bert(input_ids,
                           attention_mask = attention_mask,
                           token_type_ids = token_type_ids,
                           position_ids = position_ids,
                           head_mask = head_mask,
                           inputs_embeds = inputs_embeds)
        sequence_output = outputs[0]
        
#         print(f'[hoho], input_ids: {input_ids.shape}')
#         print(f'[hoho], input_token_starts: {input_token_starts.shape}')
#         print(f'[hoho], sequence_output: {sequence_output.shape}')
        
        origin_sequence_output = [layer[starts.nonzero().squeeze(1)] for layer, starts in zip(sequence_output, input_token_starts)]
        padded_sequence_output = pad_sequence(origin_sequence_output, batch_first = True)
        padded_sequence_output = self.dropout(padded_sequence_output)
        lstm_output, _ = self.bilstm(padded_sequence_output)
        
#         print(f'[hoho] lstm_output: {lstm_output.shape}')
        
        logits = self.classifier(lstm_output)
        outputs = (logits,)
        if labels is not None:
            loss_mask = labels.gt(-1)
            loss = self.crf(logits, labels, loss_mask) * (-1)
            
#             print(f'crf loss： {loss}')
            
            outputs = (loss,) + outputs
            
        return outputs
        

In [14]:
### hoho
test_tensor = torch.tensor([1, 1, 1, 1, 0, 0])
test_tensor.nonzero().squeeze(1)

tensor([0, 1, 2, 3])

In [30]:
### metrics.py

import os
import logging

def get_entities(seq):
    """
    Gets entities from sequence.

    Args:
        seq (list): sequence of labels.

    Returns:
        list: list of (chunk_type, chunk_start, chunk_end).

    Example:
        seq = ['B-PER', 'I-PER', 'O', 'B-LOC']
        get_entities(seq)
        [('PER', 0, 1), ('LOC', 3, 3)]
    """
    
    if any(isinstance(s, list) for s in seq):
        seq = [item for sublist in seq for item in sublist + ['O']]   #即拉平数组
    prev_tag = 'O'
    prev_type = ''
    begin_offset = 0
    chunks = []
    for i, chunk in enumerate(seq + ['O']):
        tag = chunk[0]
        type_ = chunk.split('-')[-1]
        
        if end_of_chunk(prev_tag, tag, prev_type, type_):
            chunks.append((prev_type, begin_offset, i - 1))
        
        if start_of_chunk(prev_tag, tag, prev_type, type_):
            begin_offset = i
            
        prev_tag = tag
        prev_type = type_
        
    return chunks


def end_of_chunk(prev_tag, tag, prev_type, type_):
    """Checks if a chunk ended between the previous and current word.

    Args:
        prev_tag: previous chunk tag.
        tag: current chunk tag.
        prev_type: previous type.
        type_: current type.

    Returns:
        chunk_end: boolean.
    """
    chunk_end = False

    if prev_tag == 'S':
        chunk_end = True
    # pred_label中可能出现这种情形
    if prev_tag == 'B' and tag == 'B':
        chunk_end = True
    if prev_tag == 'B' and tag == 'S':
        chunk_end = True
    if prev_tag == 'B' and tag == 'O':
        chunk_end = True
    if prev_tag == 'I' and tag == 'B':
        chunk_end = True
    if prev_tag == 'I' and tag == 'S':
        chunk_end = True
    if prev_tag == 'I' and tag == 'O':
        chunk_end = True

    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    return chunk_end


def start_of_chunk(prev_tag, tag, prev_type, type_):
    """Checks if a chunk started between the previous and current word.

    Args:
        prev_tag: previous chunk tag.
        tag: current chunk tag.
        prev_type: previous type.
        type_: current type.
        
    Returns:
        chunk_start: boolean.
    """
    chunk_start = False

    if tag == 'B':
        chunk_start = True
    if tag == 'S':
        chunk_start = True

    if prev_tag == 'S' and tag == 'I':
        chunk_start = True
    if prev_tag == 'O' and tag == 'I':
        chunk_start = True

    if tag != 'O' and tag != '.' and prev_type != type_:
        chunk_start = True

    return chunk_start

def f1_score(y_true, y_pred, mode = 'dev'):
    """Compute the F1 score.

    The F1 score can be interpreted as a weighted average of the precision and
    recall, where an F1 score reaches its best value at 1 and worst score at 0.
    The relative contribution of precision and recall to the F1 score are
    equal. The formula for the F1 score is::

        F1 = 2 * (precision * recall) / (precision + recall)

    Args:
        y_true : 2d array. Ground truth (correct) target values.
        y_pred : 2d array. Estimated targets as returned by a tagger.

    Returns:
        score : float.

    Example:
        y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
        y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
        f1_score(y_true, y_pred)
        0.50
    """
    
    true_entities = set(get_entities(y_true))
    pred_entities = set(get_entities(y_pred))
    nb_correct = len(true_entities & pred_entities)
    nb_pred = len(pred_entities)
    nb_true = len(true_entities)
    
    p = nb_correct / nb_pred if nb_pred > 0 else 0
    r = nb_correct / nb_true if nb_true > 0 else 0
    score = 2 * p * r / (p + r) if p + r > 0 else 0
    
    if mode == 'dev':
        return score
    else:
        f_score = {}
        for label in config.labels:
            true_entities_label = set()
            pred_entities_label = set()
            for t in true_entities:
                if t[0] == label:
                    true_entities_label.add(t)
            for p in pred_entities:
                if p[0] == label:
                    pred_entities_label.add(p)
                    
            nb_correct_label = len(true_entities_label & pred_entities_label)
            nb_pred_label = len(pred_entities_label)
            nb_true_label = len(true_entities_label)
            
            p_label = nb_correct_label / nb_pred_label if nb_pred_label > 0 else 0
            r_label = nb_correct_label / nb_true_label if nb_true_label > 0 else 0
            score_label = 2 * p_label * r_label / (p_label + r_label) if p_label + r_label > 0 else 0
            f_score[label] = score_label
        return f_score, score
    
def bad_case(y_true, y_pred, data):
    if not os.path.exists(config.case_dir):
        os.system(r"touch {}".format(config.case_dir))  # 调用系统命令行来创建文件
    output = open(config.case_dir, 'w')
    for idx, (t, p) in enumerate(zip(y_true, y_pred)):
        if t == p:
            continue
        else:
            output.write("bad case " + str(idx) + ": \n")
            output.write("sentence: " + str(data[idx]) + "\n")
            output.write("golden label: " + str(t) + "\n")
            output.write("model pred: " + str(p) + "\n")
    print("--------Bad Cases reserved !--------")

In [16]:
# hoho

# !mkdir case

y_t = [['O', 'O', 'O', 'B-address', 'I-address', 'I-address', 'O'], ['B-name', 'I-name', 'O']]
y_p = [['O', 'O', 'B-address', 'I-address', 'I-address', 'I-address', 'O'], ['B-name', 'I-name', 'O']]
sent = [['十', '一', '月', '中', '山', '路', '电'], ['周', '静', '说']]
# bad_case(y_t, y_p, sent)
get_entities(y_t)

[('address', 3, 5), ('name', 8, 9)]

In [17]:
# train.py

import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from transformers import BertTokenizer


def train_epoch(train_loader, model, optimizer, scheduler, epoch):
    model.train()
    
    train_losses = 0
    for idx, batch_samples in enumerate(tqdm(train_loader)):
        batch_data, batch_token_starts, batch_labels = batch_samples
        batch_masks = batch_data.gt(0)
        
#         print(f'[hoho], batch_data = {batch_data.shape}')
#         print(f'[hoho], batch_token_starts = {batch_token_starts.shape}')
#         print(f'[hoho], batch_labels = {batch_labels.shape}')
#         print(f'[hoho], batch_masks = {batch_masks.shape}')
        
        loss = model((batch_data, batch_token_starts), token_type_ids = None, attention_mask = batch_masks, labels = batch_labels)[0]
        train_losses += loss.item()
        
        model.zero_grad()
        loss.backward()
        
        nn.utils.clip_grad_norm_(parameters = model.parameters(), max_norm = config.clip_grad)
        optimizer.step()
        scheduler.step()
    train_loss = float(train_losses) / len(train_loader)
    
    print(f'Epoch: {epoch}, train loss: {train_loss}')
    
    
def train(train_loader, dev_loader, model, optimizer, scheduler, model_dir):
    if model_dir is not None and config.load_before:
        model = BertNER.from_pretrained(model_dir)
        model.to(cnofig.device)
        print("--------Load model from {}--------".format(model_dir))
    best_val_f1 = 0.0
    patientce_counter = 0
    for epoch in range(1, config.epoch_num + 1):
        train_epoch(train_loader, model, optimizer, scheduler, epoch)
        val_metrics = evaluate(dev_loader, model, mode = 'dev')
        val_f1 = val_metrics['f1']
        print(f"Epoch: {epoch}, dev loss: {val_metrics['loss']}, f1 score: {val_f1}")
        improve_f1 = val_f1 - best_val_f1
        if improve_f1 > 1e-5:
            best_val_f1 = val_f1
            model.save_pretrained(model_dir)
            if improve_f1 < config.patience:
                patience_counter += 1
            else:
                patience_counter = 0
        else:
            patience_counter += 1
            
        if (patience_counter >= config.patience_num and epoch > config.min_epoch_num) or epoch == config.epoch_num:
            print('Best val f1: {best_val_f1}')
            break
    print('Training Finished!')
    
    
def evaluate(dev_loader, model, mode='dev'):
    # set model to evaluation mode
    model.eval()
    if mode == 'test':
        tokenizer = BertTokenizer.from_pretrained(config.bert_model, do_lower_case = True, skip_special_tokens = True)
    id2label = config.id2label
    true_tags = []
    pred_tags = []
    sent_data = []
    dev_losses = 0

    with torch.no_grad():
        for idx, batch_samples in enumerate(dev_loader):
            batch_data, batch_token_starts, batch_tags = batch_samples
            if mode == 'test':
                sent_data.extend([[tokenizer.convert_ids_to_tokens(idx.item()) for idx in indices
                                   if (idx.item() > 0 and idx.item() != 101)] for indices in batch_data])
            batch_masks = batch_data.gt(0)  # get padding mask, gt(x): get index greater than x
            label_masks = batch_tags.gt(-1)  # get padding mask, gt(x): get index greater than x
            # compute model output and loss
            loss = model((batch_data, batch_token_starts),
                         token_type_ids = None, attention_mask = batch_masks, labels = batch_tags)[0]
            dev_losses += loss.item()
            # (batch_size, max_len, num_labels)
            batch_output = model((batch_data, batch_token_starts),
                                 token_type_ids = None, attention_mask = batch_masks)[0]   # 加label给出loss, 不加label才给出预测
            # (batch_size, max_len - padding_label_len)
            batch_output = model.crf.decode(batch_output, mask=label_masks)
            # (batch_size, max_len)
            batch_tags = batch_tags.to('cpu').numpy()
            pred_tags.extend([[id2label.get(idx) for idx in indices] for indices in batch_output])
            # (batch_size, max_len - padding_label_len)
            true_tags.extend([[id2label.get(idx) for idx in indices if idx > -1] for indices in batch_tags])

    assert len(pred_tags) == len(true_tags)
    if mode == 'test':
        assert len(sent_data) == len(true_tags)

    # logging loss, f1 and report
    metrics = {}
    if mode == 'dev':
        f1 = f1_score(true_tags, pred_tags, mode)
        metrics['f1'] = f1
    else:
        bad_case(true_tags, pred_tags, sent_data)
        f1_labels, f1 = f1_score(true_tags, pred_tags, mode)
        metrics['f1_labels'] = f1_labels
        metrics['f1'] = f1
    metrics['loss'] = float(dev_losses) / len(dev_loader)
    return metrics

In [18]:
### hoho

l1 = [('H', 0, 1), ('B', 0, 1), ('H', 0, 1), ('B', 3, 3)]
l2 = [('H', 0, 1), ('B', 0, 1)]
set(l1) & set(l2)

{('B', 0, 1), ('H', 0, 1)}

In [19]:
# utils.py

import logging


def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.
    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.
    Example:
    ```
    logging.info("Starting training...")
    ```
    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)

In [28]:
# run.py


from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers.optimization import get_cosine_schedule_with_warmup, AdamW

import warnings
warnings.filterwarnings('ignore')

def dev_split(dataset_dir):
    data = np.load(dataset_dir, allow_pickle = True)
    words = data['words']
    labels = data['labels']
    x_train, x_dev, y_train, y_dev = train_test_split(words, labels, test_size = config.dev_split_size, random_state = RANDOM_STATE)
    return x_train, x_dev, y_train, y_dev

def test():
    data = np.load(config.test_dir, allow_pickle = True)
    word_test = data['words']
    label_test = data['labels']
    test_dataset = NERDataset(word_test, label_test, config)
    print('--------Dataset Build!--------')
    test_loader = DataLoader(test_dataset, batch_size = config.batch_size, shuffle = False, collate_fn = test_dataset.collate_fn)
    
    print('--------Get Dataloader!--------')
    if config.model_dir is not None:
        model = BertNER.from_pretrained(config.model_dir)
        model.to(config.device)
        print(f'--------Load model from {config.model_dir}')
    else:
        print(f'--------No model to test!--------')
        return
    val_metrics = evaluate(test_loader, model, mode = 'test')
    val_f1 = val_metrics['f1']
    print(f"test loss: {val_metrics['loss']}, f1 score: {val_f1}")
    val_f1_labels = val_metrics['f1_labels']
    for label in config.labels:
        print(f'f1 score of {label}: {val_f1_labels[label]}')
        
def load_dev(mode):
    if mode == 'train':
        word_train, word_dev, label_train, label_dev = dev_split(config.train_dir)
    elif mode == 'test':
        train_data = np.load(config.train_dir, allow_pickle = True)
        dev_data = np.load(config.test_dir, allow_pickle = True)
        word_train = train_data['words']
        label_train = train_data['labels']
        word_dev = dev_data['words']
        label_dev = dev_data['labels']
    else:
        word_train = None
        label_train = None
        word_dev = None
        label_dev = None
    return word_train, word_dev, label_train, label_dev


def run():
#     set_logger(config.log_dir)
    print(f'device: {config.device}')
    
    processor = Processor(config)
    processor.process()
    print('--------Process Done!--------')
    
    word_train, word_dev, label_train, label_dev = load_dev('train')
    train_dataset = NERDataset(word_train, label_train, config)
    dev_dataset = NERDataset(word_dev, label_dev, config)
    print('--------Dataset Build!--------')
    
    train_size = len(train_dataset)
    train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle = True, collate_fn = train_dataset.collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size = config.batch_size, shuffle = True, collate_fn = train_dataset.collate_fn)
    print('--------Get Dataloader!--------')
    
    model = BertNER.from_pretrained(config.roberta_model, num_labels = len(config.label2id))
    model.to(config.device)
    
    if config.full_fine_tuning:
        bert_optimizer = list(model.bert.named_parameters())
        lstm_optimizer = list(model.bilstm.named_parameters())
        classifier_optimizer = list(model.classifier.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in bert_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay': config.weight_decay
            },
            {
                'params': [p for n, p in bert_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0
            },
            {
                'params': [p for n, p in lstm_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay': config.weight_decay,
                'lr': config.learning_rate * 5
            },
            {
                'params': [p for n, p in lstm_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
                'lr': config.learning_rate * 5
            },
            {
                'params': [p for n, p in classifier_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay': config.weight_decay,
                'lr': config.learning_rate * 5
            },
            {
                'params': [p for n, p in classifier_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
                'lr': config.learning_rate * 5
            },
            {
                'params': model.crf.parameters(),
                'lr': config.learning_rate * 5
            }
            
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [{'param': [p for n, p in param_optimizer]}]
    optimizer = AdamW(optimizer_grouped_parameters, lr = config.learning_rate, correct_bias = False)
    train_steps_per_epoch = train_size // config.batch_size
    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = (config.epoch_num // 10) * train_steps_per_epoch, 
                                                num_training_steps = config.epoch_num * train_steps_per_epoch)
    print('--------Start Training!---------')
    train(train_loader, dev_loader, model, optimizer, scheduler, config.model_dir)

In [21]:
run()

device: cuda
--------Process Done!--------


--------Dataset Build!--------
--------Get Dataloader!--------


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertNER: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertNER were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['bilstm.weight_hh_l0_

--------Start Training!---------


100%|██████████| 303/303 [01:15<00:00,  4.00it/s]


...

Epoch: 41, train loss: 0.4520463746766446
Epoch: 41, dev loss: 935.8647622501148, f1 score: 0.7789646672144617


100%|██████████| 303/303 [01:13<00:00,  4.13it/s]


Epoch: 42, train loss: 0.6191177360295462
Epoch: 42, dev loss: 940.8033186968635, f1 score: 0.7824302134646961


100%|██████████| 303/303 [01:13<00:00,  4.11it/s]


Epoch: 43, train loss: 0.4184116191989911
Epoch: 43, dev loss: 933.0171760110294, f1 score: 0.784217016029593
Best val f1: {best_val_f1}
Training Finished!


In [22]:
!nvidia-smi

Fri Apr 14 05:26:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    48W / 250W |   4129MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [31]:
test()

--------Dataset Build!--------
--------Get Dataloader!--------
--------Load model from /kaggle/working/experiments/clue/
--------Bad Cases reserved !--------
test loss: 842.7020728701636, f1 score: 0.7873965626989178
f1 score of address: 0.6493506493506493
f1 score of book: 0.8133333333333332
f1 score of company: 0.7917737789203084
f1 score of game: 0.8431372549019608
f1 score of government: 0.8061420345489443
f1 score of movie: 0.8039867109634552
f1 score of name: 0.8701973001038422
f1 score of organization: 0.7941176470588234
f1 score of position: 0.7699316628701595
f1 score of scene: 0.7312348668280871
